Một câu kinh điển của Go's philosophy: **Don't communicate by sharing memory; share memory by communicating.**  
Philosophy nên chắc là phức tạp, nhưng mình thì nói rút gọn là: **Hãy sử dụng Channel.**

## Channel là gì?

Channel thì mình thấy nó giống như cái ống, một goroutine đặt một cái gì vào đó, và một goroutine khác lấy nó ra  
Cái ống này có thể chứa 1 hoặc nhiều món đồ

In [ ]:
ch <- v    // SEND - Đặt v (món đồ) vào cái ống (ch - channel)
v := <-ch  // RECEIVE - Lấy món đồ ra khỏi cái ống, và gán nó vào v (lấy món đồ ra khỏi ống)

## Khởi tạo và sử dụng Channel

#### Một ví dụ về Channel

In [4]:
package main

import "fmt"

func main() {
    // Tạo một channel để chứa món đồ có kiểu string
    messages := make(chan string)
    
    // Tạo một goroutine, gửi "ping" tới cái channel đó
    go func() {
        messages <- "ping"
    }()
    
    // Ở main routine thì lấy ping ra khỏi channel và gán vào msg
    msg := <-messages
    fmt.Println(msg) // In ra "ping"
}

ping


#### Vài điểm chú ý:
- Channel phải được khởi tạo trước khi sử dụng
- Send và Receive bị block mặc định để đảm bảo đồng bộ dữ liệu giữa các goroutine

Tiện thể thì hàm `make` có thể được sử dụng để tạo:
- slice
- map
- channel

In [9]:
func main() {
    s := make([]int, 5, 10)
    // len = 5, cap = 10, initialized with zeros
    fmt.Println(s, len(s), cap(s)) // 5 10
}

[0 0 0 0 0] 5 10


In [7]:
func main() {
    m := make(map[string]int)
    m["apples"] = 5
    fmt.Println(m) // map[apples:5]
}

map[apples:5]


In [8]:
func main() {
    ch := make(chan string, 2) // buffered channel with capacity 2
    ch <- "hi"
    ch <- "there"
    fmt.Println(<-ch, <-ch) // hi there
}

hi there


## Hướng của Channel

#### Channel thì sẽ có 2 hướng, gọi là send và receive:
- Send là đặt món đồ vào cái ống
- Receive là lấy món đồ ra khỏi cái ống
- Từ vựng send/receive có thể dễ gây nhầm lẫn lúc đầu:
    - Lấy channel làm trung tâm
    - Send là gửi data tới cái channel đó
    - Receive là nhận data từ cái channel đó

In [10]:
// Send-only channel
func send(ch chan<- string) {
    ch <- "Hello"
}

// Receive-only channel
func receive(ch <-chan string) {
    msg := <-ch
    fmt.Println(msg)
}

#### Với arg của func:
- Để tránh tạo bug lúc làm, arg của func *thường* chỉ là send hoặc receive mà thôi
- Trick để nhớ send/receive:
    - Lấy channel làm trung tâm
    - Mũi tên đi vào cái channel thì là đang gửi data tới channel đó, gọi là send: **chan<-**
    - Mũi tên đi ra khỏi cái channel thì là đang lấy data ra khỏi channel đó, gọi là receive: **<-chan**

## Buffered Channels

In [11]:
func main() {
    // Tạo một buffered channel với capacity là 2
    ch := make(chan int, 2)
    
    // Có thể gửi 2 món đồ tới channel
    ch <- 1
    ch <- 2
    
    // ch <- 3
    // Gửi tiếp như vầy sẽ bị blog
    
    // Receive values
    fmt.Println(<-ch) // 1
    fmt.Println(<-ch) // 2
}

1
2


#### Buffered vs Unbuffered

| Aspect          | Unbuffered            | Buffered              |
|-----------------|-----------------------|-----------------------|
| **Creation**    | `make(chan T)`        | `make(chan T, n)`     |
| **Send blocks** | Until receiver ready  | When buffer full      |
| **Receive blocks** | Until sender ready | When buffer empty     |
| **Synchronization** | Guaranteed        | Looser coupling       |
| **Use case**    | Synchronization       | Performance, decoupling |


## Rob Pike's Channel Patterns

Theo Rob Pike: **Một generator là một function mà trả về một channel**

In [14]:
func boring(msg string) <-chan string {
    c := make(chan string)
    go func() {
        for i := 0; ; i++ {
            c <- fmt.Sprintf("%s %d", msg, i)
            time.Sleep(time.Duration(rand.Intn(1e3)) * time.Millisecond)
        }
    }()
    return c
}

func main() {
    n := 10
    c := boring("boring!")
    for i := 0; i < n; i++ {
        fmt.Printf("You say: %q\n", <-c)
    }
    fmt.Println("You're boring; I'm leaving.")
}

You say: "boring! 0"
You say: "boring! 1"
You say: "boring! 2"
You say: "boring! 3"
You say: "boring! 4"
You say: "boring! 5"
You say: "boring! 6"
You say: "boring! 7"
You say: "boring! 8"
You say: "boring! 9"
You're boring; I'm leaving.


#### Vài note ở ví dụ này:
- Hàm boring có input là một string, và trả về 1 string receive channel
- Hàm boring tạo 1 goroutine chạy vòng lặp vô hạn, send một string có format `fmt.Sprintf("%s %d", msg, i)` tới channel c
- c là một unbuffered channel, nên khi send một string ở trên vào c, phải chờ hàm main lấy ra, rồi mới send string mới tiếp được
- Trong hàm boring, có giả lập một khoảng sleep từ 0-1000ms, mô phỏng thời gian xử lí của goroutine này
- Ở hàm main - vòng lặp for, `i` có thể chạy tới một số `n` nào đó tùy ý, thì ở hàm boring sẽ send được sô các message với số lượng y hệt
- Chuyển c thành buffered channel thì kết quả vẫn thế

In [18]:
func boring(msg string) <-chan string {
    c := make(chan string, 4)
    go func() {
        for i := 0; ; i++ {
            c <- fmt.Sprintf("%s %d", msg, i)
            time.Sleep(time.Duration(rand.Intn(1e3)) * time.Millisecond)
        }
    }()
    return c
}

func main() {
    n := 10
    c := boring("boring!")
    for i := 0; i < n; i++ {
        fmt.Printf("You say: %q\n", <-c)
    }
    fmt.Println("You're boring; I'm leaving.")
}

You say: "boring! 0"
You say: "boring! 1"
You say: "boring! 2"
You say: "boring! 3"
You say: "boring! 4"
You say: "boring! 5"
You say: "boring! 6"
You say: "boring! 7"
You say: "boring! 8"
You say: "boring! 9"
You're boring; I'm leaving.


#### Một ví dụ khác:

In [20]:
type Request struct {
    args        []int
    resultChan  chan int
}

func sum(a []int) int {
    s := 0
    for _, v := range a {
        s += v
    }
    return s
}

func server(requests chan *Request) {
    for req := range requests {
        go func(req *Request) {
            req.resultChan <- sum(req.args)
        }(req)
    }
}

func main() {
	// Channel for incoming requests
	requests := make(chan *Request)

	// Start server in background
	go server(requests)

	// Test case 1
	req1 := &Request{
		args:       []int{1, 2, 3, 4, 5},
		resultChan: make(chan int),
	}
	requests <- req1
	fmt.Println("Result 1:", <-req1.resultChan) // Expected: 15

	// Test case 2
	req2 := &Request{
		args:       []int{10, 20, 30},
		resultChan: make(chan int),
	}
	requests <- req2
	fmt.Println("Result 2:", <-req2.resultChan) // Expected: 60

	// Test case 3
	req3 := &Request{
		args:       []int{},
		resultChan: make(chan int),
	}
	requests <- req3
	fmt.Println("Result 3:", <-req3.resultChan) // Expected: 0
}

Result 1: 15
Result 2: 60
Result 3: 0


#### Điều chú ý ở đây:
- Hàm server có vòng lặp for...range `for req := range requests` là để nghe requests mãi mãi
- Trong hàm main, khi `go server(requests)` được chạy, hàm server sẽ lắng nghe requests mãi mãi cho tới khi hàm main kết thúc
- Hàm server lắng nghe 3 request riêng biệt req1, req2, req3, và in ra kết quả
- Khi hàm main kết thúc thì goroutine mới kết thúc

## Đóng một Channel

In [25]:
func main() {
    jobs := make(chan int, 5)
    done := make(chan bool)
    
    go func() {
        for {
            j, more := <-jobs
            if !more {
                fmt.Println("received all jobs")
                done <- true
                return
            }
            fmt.Println("received job", j)
        }
    }()
    
    for j := 1; j <= 3; j++ {
        jobs <- j
        fmt.Println("sent job", j)
    }
    close(jobs)
    fmt.Println("sent all jobs")
    
    <-done
}

sent job 1
sent job 2
sent job 3
sent all jobs
received job 1
received job 2
received job 3
received all jobs


#### Vài điểm ở ví dụ trên:
- Ban đầu, chúng ta tạo 2 channel:
    - jobs để xử lí công việc
    - done để báo xử lí xong
- Khi lấy data ra khỏi channel, thực ra có thể kiểm tra channel đã close chưa ` j, more := <-jobs`:
    - j là lấy data ra khỏi channel jobs
    - còn more là để check xem channel đã close chưa
- `close(jobs)` là để đóng channel jobs
- Khi channel đã close, more là false, gửi true đến done, lấy value ra khỏi done để kết thúc hàm main
- `<-done` ở đây để giữ hàm main, comment lại và thử sẽ thấy kết quả như ở dưới

In [26]:
func main() {
    jobs := make(chan int, 5)
    done := make(chan bool)
    
    go func() {
        for {
            j, more := <-jobs
            if !more {
                fmt.Println("received all jobs")
                done <- true
                return
            }
            fmt.Println("received job", j)
        }
    }()
    
    for j := 1; j <= 3; j++ {
        jobs <- j
        fmt.Println("sent job", j)
    }
    close(jobs)
    fmt.Println("sent all jobs")
    
    // <-done
}

sent job 1
sent job 2
sent job 3
sent all jobs


**Không có receive nào được in ra. Vì sao????**   
Vì hàm main không bị block nữa mà kết thúc luôn

`<-done` có nghĩa là:  
- Chờ cho tới khi có data vào done channel và lấy nó ra (chờ sender gửi data vào channel để receive)
- Nếu chưa có data từ sender, main goroutine bị block ở đây
- Hàm main ko thể tiếp túc nếu chưa có `done <- true` ở 1 chỗ nào đó

**Thường thì người ta sử dụng WaitGroup:**

In [30]:
func main() {
    jobs := make(chan int, 5)
    var wg sync.WaitGroup

    // Worker goroutine
    wg.Add(1)
    go func() {
        defer wg.Done() // mark this goroutine as finished when function ends
        for j := range jobs {
            fmt.Println("received job", j)
        }
        fmt.Println("received all jobs")
    }()

    // Send jobs
    for j := 1; j <= 3; j++ {
        jobs <- j
        fmt.Println("sent job", j)
    }
    close(jobs)
    fmt.Println("sent all jobs")

    // Wait for worker to finish
    wg.Wait()
}


sent job 1
sent job 2
sent job 3
sent all jobs
received job 1
received job 2
received job 3
received all jobs


## Range Over Channels

Như đã giới thiệu sơ ở trên, có thể dùng `for...range` để nhận value cho tới khi đóng channel, cho dù close trước vòng lặp

In [32]:
func main() {
    queue := make(chan string, 2)
    queue <- "one"
    queue <- "two"
    close(queue)
    
    for elem := range queue {
        fmt.Println(elem)
    }
}

one
two


## Common Pitfalls - vài cạm bẫy phổ biến

### 1. Deadlock

In [33]:
// Deadlock - all goroutines blocked
func main() {
    ch := make(chan int)
    ch <- 42  // Blocks forever, no receiver
    fmt.Println(<-ch)
}

fatal error: all goroutines are asleep - deadlock!

goroutine 1 [chan send]:
main.main()
	 [[ Cell [33] Line 4 ]] /tmp/gonb_e1c925eb/main.go:55 +0x36
exit status 2


#### Vài chú ý ở đoạn code trên:
- Thoạt nhìn thì thấy có cả send lẫn receive cho channel này
- Nhưng ch là 1 unbuffered channel, nên mọi thứ sẽ bị block cho đến khi sender lẫn receiver sẵn sàng
- Khi send (`ch <- 42`) thì main goroutine sẽ bị block cho đến khi 1 goroutine khác receive
- Tương tự như giải thích về `done` ở trên, receive (`<-ch`) cũng sẽ bị block cho đến khi 1 goroutine khác send (code ko chạy tới được dòng này)
- Do đó, tại dòng ch <- 42, main goroutine đã bị block vĩnh viễn

#### Option 1: tạo thêm goroutine trong main  
Điều này ok vì goroutine là bình đẳng, 1 goroutine trong main goroutine cũng chạy đồng thời (concurrent) với nhau

In [50]:
func main() {
    ch := make(chan int)
    go func() {
        ch <- 42 // sender runs in its own goroutine
    }()
    fmt.Println(<-ch) // receiver
}

42


#### Option 2: dùng buffered channel  
Do là buffered channel nên đoạn code sẽ không bị block  
Đây là điểm khác nhau giữa unbuffered channel, và buffered channel có capacity là 1  
- Đối với unbuffered channel:
    - send bị block cho đến khi receiver sẵn sàng
    - receive bị block cho đến khi sender sẵn sàng
- Đối với buffered channel:
    - send bị block khi buffer đầy
    - receive bị block khi buffer rỗng

In [39]:
func main() {
    ch := make(chan int, 1) // buffer size 1

    ch <- 42                // succeeds immediately, no receiver yet
    fmt.Println(<-ch)       // later receive
}

42


### 2. Goroutine Leak  
Có receive mà không có send

In [43]:
// Leak - goroutine runs forever
func leak() {
    ch := make(chan int)
    go func() {
        val := <-ch  // Blocks forever if ch never receives
        fmt.Println(val)
    }()
    // Function returns, goroutine still running
}

#### Option 1: thêm sender

In [44]:
func notLeak() {
    ch := make(chan int)

    go func() {
        val := <-ch
        fmt.Println(val)
    }()

    ch <- 42 // sender ensures goroutine unblocks
}

#### Option 2: nhớ close channel

In [45]:
func notLeak() {
    ch := make(chan int)

    go func() {
        val, ok := <-ch
        if !ok {
            fmt.Println("channel closed, exiting goroutine")
            return
        }
        fmt.Println(val)
    }()

    close(ch)
}

#### Option 3: thêm context

In [46]:
func notLeak() {
    ctx, cancel := context.WithCancel(context.Background())
    defer cancel()

    ch := make(chan int)

    go func() {
        select {
        case val := <-ch:
            fmt.Println(val)
        case <-ctx.Done():
            fmt.Println("canceled")
            return
        }
    }()

    // when not sending, cancel will prevent the leak
}

### 3. Race on Channel Variable

In [47]:
// Race condition
var ch chan int

func main() {
    go func() {
        ch = make(chan int)  // Race!
    }()
    go func() {
        ch <- 42  // Race!
    }()
}

## Best practices

### 1. Ownership  
- goroutine nào tạo channel thì nên close channel ở goroutine đó
- Nên truyền channel như 1 parameter để làm rõ ownership

### 2. Directional Channels  
Nên truyền rõ ràng hướng của channel (send hay receive)

In [48]:
// ✅ Clear intent
func worker(jobs <-chan int, results chan<- int) {
    for j := range jobs {
        results <- j * 2
    }
}

### 3. Select for Non-blocking
Một kĩ thuật dùng select, nếu không có send gì thì nhảy vào default, đoạn code sẽ không bị block

In [ ]:
// ✅ Non-blocking send
select {
case ch <- value:
    // Sent successfully
default:
    // Channel was full
}